# LLM Optimization Modelling Experiment

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [90]:
problem = '''You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7'''

## 2. Generate the mathematical model

In [109]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [110]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Water Flow Maximization

**Parameters:**

* **T:** Number of time steps (T = 10)
* **N:** Number of nodes (N = 7; A, B, C, D, E, F, G)
* **P:** Number of pipes (P = 12)
* **C<sub>ij</sub>:** Capacity of pipe connecting node i and j (given in problem statement)
* **f(t):** Water inflow function at point A (f(t) = max(-4/30t^3+t^2-0.234t+3, 0))
* **M:** Large enough number for big-M constraints (e.g., sum of all pipe capacities)

**Decision Variables:**

* **x<sub>ijt</sub>:** Flow through pipe connecting node i and j at time step t 
* **y<sub>ij</sub>:** Binary variable, 1 if pipe ij is activated, 0 otherwise
* **r<sub>it</sub>:** Amount of water released from node i at time step t

**Objective Function:**

Maximize the total water outflow at point G over all time steps:

```
Maximize: ∑<sub>t=1</sub><sup>T</sup> x<sub>EGt</sub>
```

**Constraints:**

1. **Flow balance at each node:** For all nodes i ∈ N and time steps t ∈ T:
    * inflow + previous time step outflow = outflow + release:
    * `∑<sub>j∈N</sub> x<sub>jit</sub> + ∑<sub>j∈N</sub> x<sub>ijt-1</sub> = ∑<sub>j∈N</sub> x<sub>ijt</sub> + r<sub>it</sub>`
    * For the source node A: `f(t) + ∑<sub>j∈N</sub> x<sub>Ajt-1</sub> = ∑<sub>j∈N</sub> x<sub>Ajt</sub> + r<sub>At</sub>`
    * For the sink node G: `∑<sub>j∈N</sub> x<sub>jGt</sub> = ∑<sub>j∈N</sub> x<sub>jGt+1</sub>` 

2. **Pipe capacity constraints:** For all pipes ij ∈ P and time steps t ∈ T:
    * `x<sub>ijt</sub> ≤ C<sub>ij</sub> * y<sub>ij</sub>`

3. **Pipe activation limit:** 
    * `∑<sub>ij∈P</sub> y<sub>ij</sub> = 10`

4. **Non-negativity constraints:**
    * `x<sub>ijt</sub> ≥ 0` for all i, j ∈ N and t ∈ T
    * `r<sub>it</sub> ≥ 0` for all i ∈ N and t ∈ T

5. **Initial and final conditions:**
    * `x<sub>ijt</sub> = 0` for t = 0 and t = T+1 

6. **Big-M constraints for release:**  For all nodes i ∈ N and time steps t ∈ T:
    * `r<sub>it</sub> ≤ M * (1 - ∑<sub>j∈N</sub> y<sub>ij</sub>)` (release only possible if no outgoing pipes are activated) 

This model considers the inflow function, pipe capacities, limited pipe activation, and the need for water release. The objective function aims to maximize the water reaching the final point, G. The constraints ensure flow balance, adherence to pipe capacity and activation limits, and prevent water storage at connection points. 


## 3. Generate the pyomo code

In [111]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [112]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data)
T = 10  # Number of time steps
N = ['A', 'B', 'C', 'D', 'E', 'F', 'G']  # Nodes
P = {('A', 'B'), ('A', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'D'), ('C', 'F'), ('D', 'E'), ('D', 'F'), ('E', 'G'), ('F', 'G')}  # Pipes
C = { # Capacity of pipes
    ('A', 'B'): 5,
    ('A', 'C'): 7,
    ('B', 'D'): 3,
    ('B', 'E'): 4,
    ('C', 'D'): 2,
    ('C', 'F'): 6,
    ('D', 'E'): 4,
    ('D', 'F'): 3,
    ('E', 'G'): 8,
    ('F', 'G'): 5
}
M = sum(C.values())  # Big-M value

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, T)
model.N = pyo.Set(initialize=N)
model.P = pyo.Set(initialize=P)

# Parameters
model.C = pyo.Param(model.P, initialize=C)

# Variables
model.x = pyo.Var(model.P, model.T, within=pyo.NonNegativeReals) # Flow through pipes
model.y = pyo.Var(model.P, within=pyo.Binary) # Pipe activation
model.r = pyo.Var(model.N, model.T, within=pyo.NonNegativeReals) # Water release

# Objective Function
def objective_rule(model):
    return sum(model.x[('E', 'G'), t] for t in model.T)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def flow_balance_rule(model, i, t):
    if i == 'A':
        return max(-4/30*t**3 + t**2 - 0.234*t + 3, 0) + sum(model.x[(j, 'A'), t-1] for j in model.N if (j, 'A') in model.P) == sum(model.x[('A', j), t] for j in model.N if ('A', j) in model.P) + model.r['A', t]
    elif i == 'G':
        return sum(model.x[(j, 'G'), t] for j in model.N if (j, 'G') in model.P) == sum(model.x[('G', j), t+1] for j in model.N if ('G', j) in model.P)
    else:
        return sum(model.x[(j, i), t] for j in model.N if (j, i) in model.P) + sum(model.x[(i, j), t-1] for j in model.N if (i, j) in model.P) == sum(model.x[(i, j), t] for j in model.N if (i, j) in model.P) + model.r[i, t]
model.flow_balance = pyo.Constraint(model.N, model.T, rule=flow_balance_rule)

def capacity_rule(model, i, j, t):
    return model.x[(i, j), t] <= model.C[(i, j)] * model.y[(i, j)]
model.capacity = pyo.Constraint(model.P, model.T, rule=capacity_rule)

def activation_limit_rule(model):
    return sum(model.y[i, j] for (i, j) in model.P) == 10
model.activation_limit = pyo.Constraint(rule=activation_limit_rule)

def release_rule(model, i, t):
    return model.r[i, t] <= M * (1 - sum(model.y[j, i] for j in model.N if (j, i) in model.P))
model.release = pyo.Constraint(model.N, model.T, rule=release_rule)

# Initial and Final Conditions
def initial_final_rule(model, i, j, t):
    if t == 0 or t == T+1:
        return model.x[(i, j), t] == 0
    else:
        return pyo.Constraint.Skip
model.initial_final = pyo.Constraint(model.P, model.T, rule=initial_final_rule)

# Solve
solver = pyo.SolverFactory('cbc') # Choose a suitable solver
solver.solve(model)

# Output
print("Optimal Solution:")
print("Objective Value:", pyo.value(model.obj))

print("Flow through pipes:")
for (i, j) in model.P:
    for t in model.T:
        if pyo.value(model.x[(i, j), t]) > 0:
            print(f"Pipe ({i}, {j}) at time {t}: {pyo.value(model.x[(i, j), t])}")

print("Pipe activation:")
for (i, j) in model.P:
    if pyo.value(model.y[(i, j)]) == 1:
        print(f"Pipe ({i}, {j}) activated")

print("Water release:")
for i in model.N:
    for t in model.T:
        if pyo.value(model.r[i, t]) > 0:
            print(f"Node {i} at time {t}: {pyo.value(model.r[i, t])}")
```

This code defines the model, parameters, variables, objective function, and constraints as specified in the problem statement. The sample data for the pipe capacities is provided in the `C` dictionary, and the inflow function `f(t)` is implemented directly in the `flow_balance_rule`. You can replace this sample data with your actual data. The code then uses the CBC solver to find the optimal solution and prints the results.


## 4. Input problem data and try running the generated code

In [113]:
import pyomo.environ as pyo

# Sample Data (Replace with actual data)
T = 10  # Number of time steps
N = ['A', 'B', 'C', 'D', 'E', 'F', 'G']  # Nodes
P = {('A', 'B'), ('A', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'D'), ('C', 'F'), ('D', 'E'), ('D', 'F'), ('E', 'G'), ('F', 'G')}  # Pipes
C = {
    ('A', 'B'): 5,
    ('A', 'C'): 3,
    ('A', 'F'): 2,
    ('B', 'C'): 4,
    ('B', 'D'): 2,
    ('B', 'E'): 1,
    ('C', 'E'): 3,
    ('C', 'F'): 2,
    ('D', 'E'): 4,
    ('E', 'G'): 6,
    ('F', 'B'): 1,
    ('F', 'G'): 5,
}

M = sum(C.values())  # Big-M value

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, T)
model.N = pyo.Set(initialize=N)
model.P = pyo.Set(initialize=P)

# Parameters
model.C = pyo.Param(model.P, initialize=C)

# Variables
model.x = pyo.Var(model.P, model.T, within=pyo.NonNegativeReals) # Flow through pipes
model.y = pyo.Var(model.P, within=pyo.Binary) # Pipe activation
model.r = pyo.Var(model.N, model.T, within=pyo.NonNegativeReals) # Water release

# Objective Function
def objective_rule(model):
    return sum(model.x[('E', 'G'), t] for t in model.T)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def flow_balance_rule(model, i, t):
    if i == 'A':
        return max(-4/30*t**3 + t**2 - 0.234*t + 3, 0) + sum(model.x[(j, 'A'), t-1] for j in model.N if (j, 'A') in model.P) == sum(model.x[('A', j), t] for j in model.N if ('A', j) in model.P) + model.r['A', t]
    elif i == 'G':
        return sum(model.x[(j, 'G'), t] for j in model.N if (j, 'G') in model.P) == sum(model.x[('G', j), t+1] for j in model.N if ('G', j) in model.P)
    else:
        return sum(model.x[(j, i), t] for j in model.N if (j, i) in model.P) + sum(model.x[(i, j), t-1] for j in model.N if (i, j) in model.P) == sum(model.x[(i, j), t] for j in model.N if (i, j) in model.P) + model.r[i, t]
model.flow_balance = pyo.Constraint(model.N, model.T, rule=flow_balance_rule)

def capacity_rule(model, i, j, t):
    return model.x[(i, j), t] <= model.C[(i, j)] * model.y[(i, j)]
model.capacity = pyo.Constraint(model.P, model.T, rule=capacity_rule)

def activation_limit_rule(model):
    return sum(model.y[i, j] for (i, j) in model.P) == 10
model.activation_limit = pyo.Constraint(rule=activation_limit_rule)

def release_rule(model, i, t):
    return model.r[i, t] <= M * (1 - sum(model.y[j, i] for j in model.N if (j, i) in model.P))
model.release = pyo.Constraint(model.N, model.T, rule=release_rule)

# Initial and Final Conditions
def initial_final_rule(model, i, j, t):
    if t == 0 or t == T+1:
        return model.x[(i, j), t] == 0
    else:
        return pyo.Constraint.Skip
model.initial_final = pyo.Constraint(model.P, model.T, rule=initial_final_rule)

# Solve
solver = pyo.SolverFactory('cbc') # Choose a suitable solver
solver.solve(model)

# Output
print("Optimal Solution:")
print("Objective Value:", pyo.value(model.obj))

print("Flow through pipes:")
for (i, j) in model.P:
    for t in model.T:
        if pyo.value(model.x[(i, j), t]) > 0:
            print(f"Pipe ({i}, {j}) at time {t}: {pyo.value(model.x[(i, j), t])}")

print("Pipe activation:")
for (i, j) in model.P:
    if pyo.value(model.y[(i, j)]) == 1:
        print(f"Pipe ({i}, {j}) activated")

print("Water release:")
for i in model.N:
    for t in model.T:
        if pyo.value(model.r[i, t]) > 0:
            print(f"Node {i} at time {t}: {pyo.value(model.r[i, t])}")

(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
ERROR: Rule failed for Param 'C' with index ('A', 'F'): KeyError: "Index
'('A', 'F')' is not valid for indexed component 'C'"
ERROR: Constructing component 'C' from data=None failed:
        KeyError: "Index '('A', 'F')' is not valid for indexed component 'C'"


KeyError: "Index '('A', 'F')' is not valid for indexed component 'C'"

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [114]:
print(response.text)

## Mathematical Optimization Model for Water Flow Maximization

**Parameters:**

* **T:** Number of time steps (T = 10)
* **N:** Number of nodes (N = 7; A, B, C, D, E, F, G)
* **P:** Number of pipes (P = 12)
* **C<sub>ij</sub>:** Capacity of pipe connecting node i and j (given in problem statement)
* **f(t):** Water inflow function at point A (f(t) = max(-4/30t^3+t^2-0.234t+3, 0))
* **M:** Large enough number for big-M constraints (e.g., sum of all pipe capacities)

**Decision Variables:**

* **x<sub>ijt</sub>:** Flow through pipe connecting node i and j at time step t 
* **y<sub>ij</sub>:** Binary variable, 1 if pipe ij is activated, 0 otherwise
* **r<sub>it</sub>:** Amount of water released from node i at time step t

**Objective Function:**

Maximize the total water outflow at point G over all time steps:

```
Maximize: ∑<sub>t=1</sub><sup>T</sup> x<sub>EGt</sub>
```

**Constraints:**

1. **Flow balance at each node:** For all nodes i ∈ N and time steps t ∈ T:
    * inflow + previou

In [115]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data)
T = 10  # Number of time steps
N = ['A', 'B', 'C', 'D', 'E', 'F', 'G']  # Nodes
P = {('A', 'B'), ('A', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'D'), ('C', 'F'), ('D', 'E'), ('D', 'F'), ('E', 'G'), ('F', 'G')}  # Pipes
C = { # Capacity of pipes
    ('A', 'B'): 5,
    ('A', 'C'): 7,
    ('B', 'D'): 3,
    ('B', 'E'): 4,
    ('C', 'D'): 2,
    ('C', 'F'): 6,
    ('D', 'E'): 4,
    ('D', 'F'): 3,
    ('E', 'G'): 8,
    ('F', 'G'): 5
}
M = sum(C.values())  # Big-M value

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, T)
model.N = pyo.Set(initialize=N)
model.P = pyo.Set(initialize=P)

# Parameters
model.C = pyo.Param(model.P, initialize=C)

# Variables
model.x = pyo.Var(model.P, model.T, within=pyo.NonNegativeReals) # Flow through pipes
model.y = pyo.Var(model.P, within=pyo.Binary) # Pipe activation
model.r = pyo.Var(model.N, model.T, within=pyo.NonNegativeReals) # Water release

# Objective Fu